In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm
from math import factorial

In [13]:
# Hyper parameters
EPISODES = 2 # times every setting is replicated
# random.seed(1)
WriteFile = False
FileAdd = 'testPoolCentralSimulation.csv'
WarmTime = 500
RunTime = 3000

In [15]:
# set the parameters
# get the number of states S
'''
A: number of areas
M: number of total bikes
S: number of total states
Pij: transfering possibility matrix
Beta: broken rate
ArrMtx: arrival rates of each area
Theta: moving rate
Mu: fix rate
RhoMtx: matrix of ride rates
N: number of fix servers
'''

A = 2
M = 2
S = int(factorial(A+A-1+A**2+M)/factorial(A+A-1+A**2)/factorial(M)) * A
print(S)
Pij = [[0.3, 0.7],
       [0.7, 0.3]]
Beta = 0.3
ArrLst = [5.0, 5.0]
Theta = 10.0
Mu = 2.0
RhoMtx = [[1.0, 1.0], 
          [1.0, 1.0]]
N = 2

72


In [18]:
a = list(range(4))
heapq.heapify(a)
t = []
for i in a:
    if i == 2: continue
    else: t.append(i)
heapq.heapify(t)
heapq.heappop(t)

0

In [20]:
# simulate the distributed model
# with maintaining truck

class Model():
    '''
    This is the distributed model
    '''
    # initiate the parameters in this function
    def __init__(self):     
        self.timeLimit = WarmTime + RunTime
        self.areas = list(range(A))
        self.epi = 0
        
    def reset(self):
        self.T = 0 # time cursor
        self.formerT = 0
        self.servedCustomers = 0 # number of customer served during runtime in an episode

        self.state1 = [int(M/A)]*A + [0] * A  + [0] # Nis, Bis
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.stateRecord = self.state1[:-1] + self.state2[0] + self.state2[1] + [self.state1[-1]]
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        heapq.heappush(self.scheduler, [random.expovariate(Theta), 2, 0, 1])
        return self.state1, self.state2, self.T
    
    def getRecord(self):
        result = [self.epi]
        result += self.stateRecord
        result.append((self.T - self.formerT)/(WarmTime+RunTime))
        self.formerT = self.T
        self.stateRecord = self.state1[:-1] + self.state2[0] + self.state2[1] + [self.state1[-1]]
        return result
                       
    def setPerformance(self):
        if self.T>WarmTime: 
            self.servedCustomers += 1
            #print(self.servedCustomers)

    def simulate(self):
        if WriteFile:
            with open(FileAdd, 'w') as fout:
                writer = csv.writer(fout)
                for i in range(EPISODES):
                    self.reset()
                    self.epi = i
                    while self.T <= self.timeLimit:
                        # print(self.T)
                        self.stepForward()
                        #if self.T > WarmTime:
                        writer.writerow(self.getRecord())
        else:
            for i in range(EPISODES):
                self.reset()
                while self.T <= self.timeLimit:
                    self.stepForward()

        return self.servedCustomers / (self.T-WarmTime)
                
    def getRidingTime(self, s, t):
        rowS, colS, rowT, colT = s//A, s%A, t//A, t%A
        if s==t: r = 0.5
        else: r = abs(rowS-rowT) + abs(colS-colT)
        return random.expovariate(r)
    
    def pop2(self):
        self.s = []
        for e in self.scheduler:
            if e[1] == 2: continue
            else: self.s.append(e)
        heapq.heapify(self.s)
        self.scheduler = self.s

    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(Theta)
            next_time += self.T
            start, end = self.terminal, (self.terminal+1)%A
            #print('add event 2')
        else:
            next_time = random.expovariate(Mu) 
            if self.state1[A+self.terminal] < N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = self.terminal, self.terminal
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        heapq.heappop(self.scheduler)
        if random.random()<Beta:
            #if self.state1[-1] == self.terminal and self.F != []: 
            if self.state1[-1] == self.terminal: 
                # delete next truck move event
                self.pop2()
                self.addEvent(3)
            self.state1[A+self.terminal] += 1
        else:
            self.state1[self.terminal] += 1
    def truckArr(self):
        heapq.heappop(self.scheduler)
        broken = self.state1[A+self.terminal]
        if broken > 0: # 给坏车们逐一确定维修好的时间
            self.state1[A+self.terminal] = 0 
            self.state1[-1] = self.terminal
            for i in range(broken):
                self.addEvent(3)
                self.state1[A+self.terminal] += 1
        else:
            self.addEvent(2)
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[A+self.terminal] <= N: heapq.heappop(self.F)
        self.state1[A+self.terminal] -= 1
        self.state1[self.terminal] += 1
        if self.state1[A+self.terminal] == 0:
            self.addEvent(2)
    def cusArr(self):
        #print(self.state1, self.state2)
        #print('------------------------')
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            self.setPerformance()
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)

    def stepForward(self):
        event = self.scheduler[0]
        #print(event)
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: truck arrives
         3: a bike is fixed
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.truckArr() # 维修车到达
        elif self.kind == 3:
            self.repair() # 修好一辆车
        else:# 顾客到达
            self.cusArr() #顾客到达

        return self.state1, self.state2, self.T



env = Model()
%time env.simulate()

# for r in tqdm(range(1000, 1001)):
#     for i in range(1000, 1001):
#         env.R['lambda0']=env.R['lambda1']=env.R['lambda2']=env.R['lambda3'] = r
#         env.N = i
#         result.append([r, i, env.simulate(IF_WRITEFILE)])


# test the influence of certain parameter
# for i in tqdm(range(1,100)):
#     env.N = i
#     result.append(env.simulate(IF_WRITEFILE))
# plt.plot(result)
# plt.show()

CPU times: user 405 ms, sys: 5.33 ms, total: 410 ms
Wall time: 512 ms


1.4473177762292706

In [9]:
names=['epi', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)
r[['a','b']] = r[['a','b']].astype(int)
r.t = r.t/(RunTime+WarmTime)
r.head()

,epi,a,b,c,d,e,f,g,h,i,t
0,0,2,2,0,0,0,0,0,0,-1,1.699077e-09
1,0,2,2,0,0,0,0,0,0,-1,1.436092e-08
2,0,1,2,0,0,0,1,0,0,-1,2.523935e-10
3,0,0,2,0,0,0,2,0,0,-1,3.067908e-09
4,0,0,2,0,0,0,2,0,0,-1,4.717089e-09


In [11]:
re = []
for i in range(EPISODES):
    re.append(r[r.epi == i].drop(columns=['epi']).groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum())
re[20]

IndexError: list index out of range

In [8]:
%store -r disState
%store -r disPorState
State, portionState = disState, disPorState

In [119]:
names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)

r['sep'] = r.t.diff()
r = r[1:]
r[['a','b']] = r[['a','b']].astype(int)
r = r.drop(columns=['t'])
#r.head()
t = r.groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum()

t.sep = t.sep/RunTime
t.sort_values('sep', ascending=False)
t.head()

sep
a b c d e f g h i          
0 0 0 0 0 0 0 0 4  0.000118
              1 3  0.000547
              2 2  0.000874
              3 1  0.000775
              4 0  0.000247

In [133]:
names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)
r[['a','b']] = r[['a','b']].astype(int)
#r.head()
r.groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum()

t
a b c d e f g h i            
0 0 0 0 0 0 0 0 4  108.409006
              1 3   44.467011
              2 2   32.636782
              3 1   14.866966
              4 0    7.221573
...                       ...
3 0 0 0 0 1 0 0 0    1.005506
        1 0 0 0 0    1.694966
      1 0 0 0 0 0    0.551165
    1 0 0 0 0 0 0    0.588252
  1 0 0 0 0 0 0 0    0.471877

[296 rows x 1 columns]